# Description

Ingram 2019 reports expressions for the errors of real and imaginary parts as derived in Bendat and Piersol 2010:

$$\sigma_{Re[C]} = \sqrt{ \frac{P_1 P_2 + (Re[C])^2 - (Im[C])^2}{2N} }$$

$$\sigma_{Im[C]} = \sqrt{ \frac{P_1 P_2 - (Re[C])^2 + (Im[C])^2}{2N} }$$

Here, we assume to have an ensamble or set of N lightcurves in two different energy bands, lc_list1 and lc_list2. The Cross-Spectrum C is the average of the products of FFT[LC1] and FFT[LC2]\*, where LC1 and LC2 are single realization or elements of the lc_list1 and lc_list2 ensambles and \* denotes complex conkucation:

$$C = \frac{1}{N} \sum_{i=1}^N FFT[LC_{1,i}]FFT[LC_{2,i}]^{*}$$

Similarly, $P_{1}$ and $P_{2}$ are average power spectra on the same ansamble of N lightcurves in the two energy bands.

$saturnx$ implements power spectra and cross-spectra list, allowing then to compute average power and cross-spectra. Such averages can be computed at different "levels", i.e., for example, you can compute the average of all the elements of a list or sorting such elements according to certain criteria (like belonging for a specific GTI) to produce a new list. 
The previous error formulas require average Fourier products to be computed (power spectra and cross-spectrum). As it would be convenient computing the cross-spectrum error without computing the corresponding (from the same ensable) power spectra, I want to verify if the errors computed on an ensamble of N elements using the formulas above is equal to the errors obtained by weighted average of M sub-ensambles.

Let's say I have two simmultaneous lightcurves in two energy bands (soft, hard): tot_lc1 and tot_lc2. I will split these two energy bands into M GTIs first and then into N segments. Each GTI will contain a number $w_g$ of segments (w stands for weight and g for GTI index). I will compute cross-spectra and errors in two ways: 1) On the ensamble of N elements using the formulas above and 2) performing a weighted average of M cross-spectra and errors computed on the M sub-ensamble each of $w_g$ elements, with $g = 0, 1, ... , M-1$. Finally, I will compare the results

# Importing stuff

In [1]:
import sys
sys.path.append('/Volumes/Samsung_T5/saturnx')

import numpy as np
from scipy.fftpack import fftfreq, fft

import matplotlib.pyplot as plt

from saturnx.core.lightcurve import Lightcurve, LightcurveList
from saturnx.core.gti import Gti
from saturnx.core.power import PowerSpectrum
from saturnx.utils.time_series import poi_events

%matplotlib inline

# Initializing lightcurves

In [7]:
def fake_white_noise_lc(tres=0.001,nbins=50000,cr=5,low_en=0.5,high_en=10):
    events = poi_events(tres=tres,nbins=nbins,cr=cr)
    time_bin_edges = np.linspace(0,nbins*tres,nbins+1,dtype=np.double)
    time_bins_center = np.linspace(0+tres/2.,nbins*tres-tres/2.,nbins,dtype=np.double)
    hist, dummy = np.histogram(events,time_bin_edges)
    notes = {}
    notes['STEF1'] = 'This is a test note'    
    meta_data = {}
    meta_data['MISSION'] = 'NICER'
    low_en, high_en = 0.5,10
    lc = Lightcurve(time_array = time_bins_center,count_array = hist,
                    low_en=low_en,high_en=high_en,
                    notes=notes, meta_data = meta_data)
    return lc

In [8]:
soft_tot_lc = fake_white_noise_lc(low_en=0.5,high_en=2)
hard_tot_lc = fake_white_noise_lc(low_en=2,high_en=10)

In [11]:
print('tres:',soft_tot_lc.tres)
print('texp:',soft_tot_lc.texp)
print('n_bins:',len(soft_tot_lc))
print('LC type',type(soft_tot_lc))

tres: 0.001
texp: 50.0
n_bins: 50000
LC type <class 'saturnx.core.lightcurve.Lightcurve'>


# Initializing GTI

In [12]:
tstarts = [0.001,17,30]
tstops  = [16,28,50]
gti = Gti(start_array = tstarts, stop_array = tstops)

In [13]:
gti

,start,stop,dur,gap
0,0.001,16,15.999,0.0
1,17.000,28,11.000,1.0
2,30.000,50,20.000,2.0


# Splitting Lightcurves

In [16]:
soft_gti_lc_list = soft_tot_lc.split(gti)
hard_gti_lc_list = hard_tot_lc.split(gti)

===> Splitting GTI
===> Splitting GTI


In [17]:
tseg = 3 # Duration of time segment in seconds
soft_seg_lc_list = soft_gti_lc_list.split(tseg)
hard_seg_lc_list = hard_gti_lc_list.split(tseg)
# soft_seg_lc_list = soft_lc.split(tseg)
# hard_seg_lc_list = hard_lc.split(tseg)

===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment


In [20]:
# Verifying the lightcurve has been correctly split
for lc in soft_gti_lc_list:
    print(lc.time.iloc[0],lc.time.iloc[-1],lc.tres,lc.texp)

0.0014999999999999998 15.999499999999998 0.001 15.999
17.000499999999995 27.999499999999994 0.001 11.0
30.00049999999999 49.9995 0.001 20.0


# Extracting weights (number of segments per GTI)

In [21]:
weights = []
first = True
counter = 0
for lc in soft_seg_lc_list:
    if first:
        first=False
        gti_index = lc.meta_data['GTI_INDEX']
        
    current_gti_index = lc.meta_data['GTI_INDEX']
    if current_gti_index == gti_index:
        counter += 1
    else:
        gti_index = current_gti_index
        weights += [counter]
        counter=1
weights += [counter]
weights = np.array(weights)
print(weights)

[5 3 6]


# Defining cross-spectrum computation functions

In [47]:
def comp_cross(lc1,lc2):
    fft1 = fft(lc1.counts.to_numpy())
    fft2 = fft(lc2.counts.to_numpy())
    cross = np.multiply(fft1,np.conj(fft2))
    return cross

def comp_cross_err(pw1,pw2,cross,n):
    '''
    Following Bendat & Piersol 2010
    '''
    re2 = (np.real(cross))**2
    im2 = (np.imag(cross))**2
    dre = np.sqrt( (pw1.power*pw2.power + re2 - im2) / 2 / n )
    dim = np.sqrt( (pw1.power*pw2.power - re2 + im2) / 2 / n )
    return dre,dim

def comp_cross_ave(lc_list1,lc_list2):
    '''
    Computes average Cross-Spectrum and errors over an ensamble 
    '''
    
    assert lc_list1 == lc_list2
    
    first = True
    for lc1, lc2 in zip(lc_list1,lc_list2):
        if first:
            first = False
            cross = comp_cross(lc1,lc2)
        else:
            cross += comp_cross(lc1,lc2)
    average_cross = cross/len(lc_list1)
    
    pw_list1 = PowerSpectrum.from_lc(lc_list1)
    pw_list2 = PowerSpectrum.from_lc(lc_list2)
    
    pw1 = pw_list1.average(norm=None)
    pw2 = pw_list2.average(norm=None)
    
    dre,dim = comp_cross_err(pw1,pw2,average_cross,len(lc_list1))
    return average_cross, dre, dim

def comp_weighted_cross_ave(cross_array,dre_array,dim_array,weights):
    '''
    Computes average Cross-Spectrum and errors via weighted average
    '''
    assert len(cross_array) == len(dre_array)
    assert len(cross_array) == len(dim_array)
    assert len(cross_array) == len(weights)
    
    ave_cross = cross_array[0]*weights[0]
    ave_dre2 = dre_array[0]**2*weights[0]
    ave_dim2 = dim_array[0]**2*weights[0]
    for i in range(1,len(cross_array)):
        ave_cross += cross_array[i]*weights[i]
        ave_dre2 += (dre_array[i]**2)*weights[i]
        ave_dim2 += (dim_array[i]**2)*weights[i]
    ave_cross = ave_cross / np.sum(weights) 
    ave_dre2 = ave_dre2 / np.sum(weights) 
    ave_dim2 = ave_dim2 / np.sum(weights) 
    
    return ave_cross, np.sqrt(ave_dre2), np.sqrt(ave_dim2)

# Computing Cross-spectrum and errors over an ensamble of N elements

In [49]:
ave_cross_from_seg, dre_from_seg, dim_from_seg = comp_cross_ave(soft_seg_lc_list, hard_seg_lc_list)

# Computing Cross-spectrum and errors via weighted average

In [50]:
# Initializing lists of segment lightcurves per GTI
soft_lc_gti_list2 = [LightcurveList() for i in range(3)]
hard_lc_gti_list2 = [LightcurveList() for i in range(3)]
for lc1,lc2 in zip(soft_seg_lc_list,hard_seg_lc_list):
    gti_index = lc1.meta_data['GTI_INDEX']
    soft_lc_gti_list2[gti_index] += [lc1]
    hard_lc_gti_list2[gti_index] += [lc2]       

In [51]:
# Checking correct initialization
for item1,item2,item3 in zip(hard_lc_gti_list2,soft_lc_gti_list2,weights):
    print(len(item1),len(item2),item3)

5 5 5
3 3 3
6 6 6


In [52]:
cross_gti = [comp_cross_ave(lc_list1,lc_list2)[0] for lc_list1,lc_list2 in zip(soft_lc_gti_list2,hard_lc_gti_list2)]
dre_gti   = [comp_cross_ave(lc_list1,lc_list2)[1] for lc_list1,lc_list2 in zip(soft_lc_gti_list2,hard_lc_gti_list2)]
dim_gti   = [comp_cross_ave(lc_list1,lc_list2)[2] for lc_list1,lc_list2 in zip(soft_lc_gti_list2,hard_lc_gti_list2)]

In [53]:
ave_cross_from_gti, dre_from_gti, dim_from_gti = comp_weighted_cross_ave(cross_gti,dre_gti,dim_gti,weights=weights)

# Comparing results

In [54]:
precision = 1e-12
assert np.allclose(ave_cross_from_seg,ave_cross_from_gti, rtol=precision, atol=precision)

In [61]:
np.mean(dre_from_gti/dre_from_seg)

1.7379611102885002

In [57]:
np.mean(dim_from_gti/dim_from_seg)

1.7376743221194062

# Comparing results for different time resolutions, gtis, and time segment

In [72]:
boundaries = sorted(np.random.randint(0,len(soft_tot_lc),size=6))
starts = [soft_tot_lc.time.iloc[boundaries[i]] for i in range(6) if i%2==0]
stops  = [soft_tot_lc.time.iloc[boundaries[i]] for i in range(6) if i%2==1]

print(starts)
print(stops)

[9.487499999999999, 18.679499999999994, 35.37349999999999]
[16.857499999999995, 25.033499999999993, 47.707499999999996]


In [92]:
ngti_array = [3,5,7,11]
norm_tseg_array = [0.1,0.3,0.49]
tres_array = [0.001,0.01,0.1,1,10]
data = []
for tres in tres_array:
    
    print('='*72)
    print('tres',tres)
    
    soft_tot_lc = fake_white_noise_lc(tres=tres,low_en=0.5,high_en=2)
    hard_tot_lc = fake_white_noise_lc(tres=tres,low_en=2,high_en=10)
    
    for ngtis in ngti_array:
        print('-'*72)
        print('ngtis:',ngtis)
        
        sec_nbins = int(len(soft_tot_lc)/ngtis)
        boundaries = []
        for g in range(ngtis):
            start_bin = np.random.randint(int(g*sec_nbins),int(g*sec_nbins+sec_nbins/2))
            stop_bin = np.random.randint(int((g*sec_nbins+sec_nbins/2)),int((g+1)*sec_nbins))
            boundaries+=[start_bin,stop_bin]
        #boundaries = sorted(np.random.randint(0,len(soft_tot_lc),size=ngtis*2))
        tstarts = [soft_tot_lc.time.iloc[boundaries[i]] for i in range(ngtis*2) if i%2==0]
        tstops  = [soft_tot_lc.time.iloc[boundaries[i]] for i in range(ngtis*2) if i%2==1]
        gti = Gti(start_array = tstarts, stop_array = tstops)
        print('min_gti_dur',np.min(gti.dur))
        
        soft_gti_lc_list = soft_tot_lc.split(gti)
        hard_gti_lc_list = hard_tot_lc.split(gti)
        
        for norm_tseg in norm_tseg_array:
            tseg = norm_tseg*np.min(gti.dur)
            print('*'*72)
            print('tseg:',tseg)
            soft_seg_lc_list = soft_gti_lc_list.split(tseg)
            hard_seg_lc_list = hard_gti_lc_list.split(tseg)
            
            weights = []
            first = True
            counter = 0
            for lc in soft_seg_lc_list:
                if first:
                    first=False
                    gti_index = lc.meta_data['GTI_INDEX']

                current_gti_index = lc.meta_data['GTI_INDEX']
                if current_gti_index == gti_index:
                    counter += 1
                else:
                    gti_index = current_gti_index
                    weights += [counter]
                    counter=1
            weights += [counter]
            weights = np.array(weights)
            
            ave_cross_from_seg, dre_from_seg, dim_from_seg = comp_cross_ave(soft_seg_lc_list, hard_seg_lc_list)
            
            # Initializing lists of segment lightcurves per GTI
            soft_lc_gti_list2 = [LightcurveList() for i in range(len(gti))]
            hard_lc_gti_list2 = [LightcurveList() for i in range(len(gti))]
            for lc1,lc2 in zip(soft_seg_lc_list,hard_seg_lc_list):
                gti_index = lc1.meta_data['GTI_INDEX']
                soft_lc_gti_list2[gti_index] += [lc1]
                hard_lc_gti_list2[gti_index] += [lc2] 
                
            cross_gti = [comp_cross_ave(lc_list1,lc_list2)[0] for lc_list1,lc_list2 in zip(soft_lc_gti_list2,hard_lc_gti_list2)]
            dre_gti   = [comp_cross_ave(lc_list1,lc_list2)[1] for lc_list1,lc_list2 in zip(soft_lc_gti_list2,hard_lc_gti_list2)]
            dim_gti   = [comp_cross_ave(lc_list1,lc_list2)[2] for lc_list1,lc_list2 in zip(soft_lc_gti_list2,hard_lc_gti_list2)]
            
            ave_cross_from_gti, dre_from_gti, dim_from_gti = comp_weighted_cross_ave(cross_gti,dre_gti,dim_gti,weights=weights)
            
            precision = 1e-12
            cond = np.allclose(ave_cross_from_seg,ave_cross_from_gti, rtol=precision, atol=precision)
            if not cond:
                print('Cross-spectra not so close')
            dreratio = np.mean(dre_from_gti/dre_from_seg)
            dimratio = np.mean(dim_from_gti/dim_from_seg)
            print('re gti/seg',dreratio)
            print('im gti/seg',dimratio)
            data += [{
                'nbins':len(soft_tot_lc),
                'tres':tres,
                'gti':gti,
                'tseg':tseg,
                'cross_close': cond,
                'dreratio':dreratio,
                'dimratio':dimratio
            }]
            print('*'*72)
        print('-'*72)
    print('='*72)

tres 0.001
------------------------------------------------------------------------
ngtis: 3
min_gti_dur 8.093999999999998
===> Splitting GTI
===> Splitting GTI
************************************************************************
tseg: 0.8093999999999998
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
re gti/seg 1.736845964624898
im gti/seg 1.7432816825030515
************************************************************************
************************************************************************
tseg: 2.428199999999999
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
re gti/seg 1.750081572297838
im gti/seg 1.7440109154971621
************************************************************************
************************************************************************
tseg: 3.966059999999999
===> Sp

re gti/seg 1.7203475384221814
im gti/seg 1.729307022751806
************************************************************************
************************************************************************
tseg: 41.8117
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
re gti/seg 1.7093495365558269
im gti/seg 1.7085459830245227
************************************************************************
------------------------------------------------------------------------
------------------------------------------------------------------------
ngtis: 5
min_gti_dur 22.899999999999977
===> Splitting GTI
===> Splitting GTI
************************************************************************
tseg: 2.289999999999998
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> 

===> Splitting Segment
re gti/seg 2.2385094909619387
im gti/seg 2.2381984760039417
************************************************************************
************************************************************************
tseg: 100.5
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
re gti/seg 2.2351871263950853
im gti/seg 2.2270484385852307
************************************************************************
************************************************************************
tseg: 164.15
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
re gti/seg 2.2275841168190507
im gti/seg 2.2356503717111145
********************

re gti/seg 2.245108379685896
im gti/seg 2.2269988948113144
************************************************************************
------------------------------------------------------------------------
------------------------------------------------------------------------
ngtis: 7
min_gti_dur 2044.0
===> Splitting GTI
===> Splitting GTI
************************************************************************
tseg: 204.4
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
re gti/seg 2.643312609020858
im gti/seg 2.6433836060468936
************************************************************************
************************************************************************
tseg: 613.1999999999999
===> Splitting Segme

re gti/seg 2.6441355005698752
im gti/seg 2.626359641463461
************************************************************************
************************************************************************
tseg: 8594.6
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
re gti/seg 2.635908351861351
im gti/seg 2.615168020832367
************************************************************************
------------------------------------------------------------------------
------------------------------------------------------------------------
ngtis: 11
min_gti_dur 2170.0
===> Splitting GTI
===> Splitting GTI
************************************************************************
tseg: 217.0
===> Splitting Segment
===> Spl

In [93]:
import pickle
with open('cross_experiment_data.pkl','wb') as outfile:
    pickle.dump(data, outfile, pickle.HIGHEST_PROTOCOL)